In [1]:
import sys
import json

sys_path = json.load(open('../../../paths.json',))['sys_path']

sys.path.append(sys_path)

from loss import *
from unet import UNet
from encoder import AutoEncoder
from multi_scale import MultiScale
from dataloader import *
from BaseModel import BaseModel
from recursive_predictions import *

from kornia.filters import spatial_gradient

import h5py

import torch.optim as optim
import time
import json

print(os.getcwd())

c:\Users\nilso\Documents\EPFL\PDM\PDM_PINN\DNN_Test\Layered\Time


In [2]:
class Model(BaseModel):
    def __init__(self, net, opt=None, sched=None, logger=None, print_progress=True, device='cuda:0'):
        super().__init__(net, opt, sched, logger, print_progress, device)

    def forward_loss(self, data):
        return

In [3]:
# # # Data
data_dir = '../../Training_Data/Moseley_EARTH_1/'
data_csv = '../../Training_Data/Moseley_Earth_1.csv'
velocity_field = '../../Training_Data/Velocity_Field_1.npy'
training_data = dataset(data_dir,data_csv,velocity_field=velocity_field)

model_names = ['L2_E500','L2_GDL_E500','L2_GDL_MAE_E500','AR_PINN_MSE_E500','PINN_MSE_E500','PINN_RES_E500']
model_types = ['unet','encoder','multiscale']

for net, model_type in  zip([UNet(in_channels=5,out_channels=1),AutoEncoder(in_channels=5,out_channels=1),MultiScale(in_channels=5)],model_types):
    for model_name in model_names:

        print(f'Predictions for :{model_type} - {model_name}')
        model = Model(net, device='cuda:0')
        model.load(import_path=f'../Training/{model_type}/results/{model_name}.pt',map_location='cuda:0')

        predictions = RecursivePredictions(net=model.net,dataset=training_data)

        # Load receivers data to obtain the coordinates
        path = f"E:/PDM_SALVUS/NoCrack/Moseley_Homogeneous/Event0000/meta.json"
        f = open(path,)
        data = json.load(f)

        # # # Parameters for the comparison
        initial_timestep = 0
        number_of_timestep_to_predict = 351 # 151 max 
        time_timestep = 0.002
        timestep_to_plot = [0,50,150,250,350]

        x_locations = pd.DataFrame.from_dict(data['forward_run_input']['output']['point_data']['receiver'])['location'].apply(pd.Series).astype(int)[0].values
        y_locations = pd.DataFrame.from_dict(data['forward_run_input']['output']['point_data']['receiver'])['location'].apply(pd.Series).astype(int)[1].values

        x_locations = np.where(x_locations == 1500, 1495, x_locations)
        y_locations = np.where(y_locations == 1500, 1495, y_locations)

        dh = 5

        # # # Names
        results_name = f"{model_type}_{model_name}.csv"
        errors_name = f"models_output/{model_type}_errors_{model_name}.npy"
        preds_name = f"models_output/{model_type}_preds_{model_name}.npy"
        trues_name = f"models_output/{model_type}_trues_{model_name}.npy"

        save_path = 'models_output/' + results_name 

        # # # Predictions
        preds, trues, errors, dict_metrics = predictions.predict_all(initial_timestep=initial_timestep,number_of_timestep_to_predict=number_of_timestep_to_predict,time_timestep=time_timestep,     timestep_to_plot=timestep_to_plot,x_locations=x_locations,y_locations=y_locations,dh=dh,model=model_type)

        # # # Save
        pd.DataFrame.from_dict(dict_metrics).to_csv(save_path)

        for filename, object in zip([errors_name,trues_name,preds_name],[errors,trues,preds]):
            np.save(filename, object) 

ep 297 in seconds 0.16

Total time for the timestep 298 in seconds 0.16

Total time for the timestep 299 in seconds 0.16

Total time for the timestep 300 in seconds 0.16

Total time for the timestep 301 in seconds 0.16

Total time for the timestep 302 in seconds 0.16

Total time for the timestep 303 in seconds 0.16

Total time for the timestep 304 in seconds 0.16

Total time for the timestep 305 in seconds 0.16

Total time for the timestep 306 in seconds 0.16

Total time for the timestep 307 in seconds 0.16

Total time for the timestep 308 in seconds 0.16

Total time for the timestep 309 in seconds 0.16

Total time for the timestep 310 in seconds 0.16

Total time for the timestep 311 in seconds 0.16

Total time for the timestep 312 in seconds 0.16

Total time for the timestep 313 in seconds 0.16

Total time for the timestep 314 in seconds 0.16

Total time for the timestep 315 in seconds 0.16

Total time for the timestep 316 in seconds 0.16

Total time for the timestep 317 in seconds 0.